# Combined Feeds
* Tiffany: <i>schedule_gtfs_dataset_names=LA Metro Bus/Railare combined/aggregated to organization_name =A County Metropolitan Transportation Authority -> both feeds have unique information, should be shown</i>
* Find other instances of this in our dataset. 

In [1]:
import geopandas as gpd
import merge_data
import pandas as pd
import yaml
from segment_speed_utils import gtfs_schedule_wrangling, helpers
from segment_speed_utils.project_vars import COMPILED_CACHED_VIEWS, PROJECT_CRS
from shared_utils import catalog_utils, portfolio_utils, publish_utils, rt_dates
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
import _operators_prep

In [4]:
all_ops = _operators_prep.load_schd_vp_df(filter_schd_both=False)

In [5]:
publish_utils.filter_to_recent_date??

Signature: publish_utils.filter_to_recent_date(df: pandas.core.frame.DataFrame) -> pandas.core.frame.DataFrame
Source:   
def filter_to_recent_date(df: pd.DataFrame) -> pd.DataFrame:
    """
    By schedule_gtfs_dataset_name, keep the most recent
    service_date that shows up in scheduled trips.
    """
    df2 = (
        df.groupby("name", group_keys=False)
        .service_date.max()
        .reset_index()
        .sort_values(["service_date", "name"], ascending=[False, True])
        .reset_index(drop=True)
        .astype({"service_date": "str"})
    )
    return df2
File:      ~/data-analyses/_shared_utils/shared_utils/publish_utils.py
Type:      function

In [14]:
def load_schd_vp() -> pd.DataFrame:
    schd_vp_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.route_schedule_vp}.parquet"

    # Drop any duplicates
    schd_vp_df = (
        pd.read_parquet(
            schd_vp_url,
            columns=[
                "schedule_gtfs_dataset_key",
                "organization_name",
                "name",
                "sched_rt_category",
                "service_date",
            ],
        )
    ).drop_duplicates()

    # Sort values so the most current row stays at the top
    schd_vp_df = schd_vp_df.sort_values(
        by=[
            "organization_name",
            "service_date",
        ],
        ascending=[True, False],
    )
    
    # Drop duplicates again
    schd_vp_df = schd_vp_df.drop_duplicates(subset=["organization_name", "name"])

    # Drop service_date
    schd_vp_df = schd_vp_df.drop(columns=["service_date"]).reset_index(drop=True)

    # Fillna rows with None.
    schd_vp_df = schd_vp_df.fillna("None")
    
    # Filter out organization_names that are None.
    schd_vp_df = schd_vp_df.loc[schd_vp_df.organization_name != "None"].reset_index(
    drop=True
)
    return schd_vp_df

In [15]:
schd_vp_df = load_schd_vp()

### Check out rows w/ no `organization_name` but a populated `name`
* There are other rows with `organization_name` filled for either same/very similar `name` values.

In [11]:
no_org_name = schd_vp_df.loc[schd_vp_df.organization_name == "None"]

In [12]:
no_org_name_gtfs = list(no_org_name.schedule_gtfs_dataset_key.unique())

In [13]:
schd_vp_df.loc[schd_vp_df.schedule_gtfs_dataset_key.isin(no_org_name_gtfs)]

,schedule_gtfs_dataset_key,organization_name,name,sched_rt_category
6,b0760015c9fcd0500c4fddd5b9bb115b,Basin Transit,Morongo Basin Schedule,schedule_only
129,dcb6fb26f2d08393f9b0b1c14dc8775c,Livermore-Amador Valley Transit Authority,Bay Area 511 Tri-Valley Wheels Schedule,schedule_and_vp
140,0d65d96d07115e28313f207d5ed0d3b2,Modoc Transportation Agency,Sage Stage Schedule,schedule_only
216,09ff99ecac281cec96e67a0890b17d39,None,Bay Area 511 Emery Express Schedule,schedule_only
217,0139b1253130b33adcd4b3a4490530d2,None,None,vp_only
218,bc039937fdadd173bd3c3edc03b7a9c9,None,Banning Pass Schedule,schedule_only
219,dcb6fb26f2d08393f9b0b1c14dc8775c,None,Bay Area 511 Tri-Valley Wheels Schedule,schedule_only
220,0d65d96d07115e28313f207d5ed0d3b2,None,Sage Stage Schedule,schedule_only
221,b0760015c9fcd0500c4fddd5b9bb115b,None,Morongo Basin Schedule,schedule_only


In [16]:
schd_vp_df.loc[schd_vp_df.organization_name.str.contains("Banning")]

,schedule_gtfs_dataset_key,organization_name,name,sched_rt_category
24,ebc783bace70899492d6206c352547d6,City of Banning,Banning Pass Schedule,schedule_and_vp


In [17]:
schd_vp_df.loc[schd_vp_df.organization_name.str.contains("Emery")]

,schedule_gtfs_dataset_key,organization_name,name,sched_rt_category
112,12a04922f8e3a4622fbefc232ff191b0,Emeryville Transportation Management Agency,Bay Area 511 Emery Go-Round Schedule,schedule_and_vp


## YML 2: Illustrating 1:1, m:1, m:m `schedule_gtfs_dataset_key` to `organization_name`

In [28]:
def df_to_yaml(
    df: pd.DataFrame, nest1_column: str, nest2_column: str, SITE_YML: str, title: str
):
    """
    Dump Pandas Dataframe to a YAML.

    Parameters:
    df (pd.DataFrame): DataFrame with 'sched_rt_category' and 'organization_name' columns.
    title (str): Title to be added at the top of the YAML file.

    Returns:
    yaml_str (str): YAML string representation of the input DataFrame.
    """
    # Initialize an empty dictionary to store the result
    result = {}

    # Iterate over unique  values in nest1_column
    for category in df[nest1_column].unique():
        # Filter the DataFrame for the current category
        category_df = df[df[nest1_column] == category]

        # Create a list of unique values in nest2_column for the current category
        organization_names = category_df[nest2_column].tolist()

        # Add the category and organization names to the result dictionary
        result[category] = organization_names

    # Save to YML
    with open(SITE_YML, "w") as f:
        f.write(f"# {title}\n\n")
        output = yaml.dump(result, default_flow_style=False)
        f.write(output)
    print("Saved to yml")

In [19]:
def count_orgs(df: pd.DataFrame, groupby_col: str, nunique_col: str) -> list:
    """
    Count the number of unique values the nunique_col
    to the groupby_col. Filter out any
    groupby_col with less than 2 unique
    values in nunique_col. Return these groupby_col values
    in a list.
    """
    agg1 = df.groupby([groupby_col]).agg({nunique_col: "nunique"}).reset_index()

    # Filter out rows with more than 1 organization_name
    agg1 = agg1.loc[agg1[nunique_col] > 1].reset_index(drop=True)

    # Grab groupby_col into a list
    multi_org_list = list(agg1[groupby_col].unique())
    return multi_org_list

In [26]:
def generate_key_org_ymls(df: pd.DataFrame):
    """
    Generate the ymls that display the relationship
    between schedule_gtfs_dataset_key to organization_name
    values.
    """
    # One `organization_name` to many `schedule_gtfs_dataset_key`
    one_org_m_keys_list = count_orgs(
        df, "organization_name", "schedule_gtfs_dataset_key"
    )
    # Filter
    one_org_m_keys_df = df.loc[df.organization_name.isin(one_org_m_keys_list)].drop(
        columns=["sched_rt_category"]
    )

    # One `schedule_gtfs_dataset_key` to many `organization_name`
    one_key_many_orgs_list = count_orgs(
        df,
        "schedule_gtfs_dataset_key",
        "organization_name",
    )

    # Filter
    one_key_many_orgs_df = df.loc[
        df.schedule_gtfs_dataset_key.isin(one_key_many_orgs_list)
    ]

    # Merge them back together. This way we can find the many schedule_gtfs_dataset_key
    # to many organization_name values.
    m1 = pd.merge(
        one_org_m_keys_df,
        one_key_many_orgs_df,
        on=["schedule_gtfs_dataset_key", "organization_name", "name"],
        how="outer",
        indicator=True,
    )

    indicator_values = {
        "left_only": "1 organization_name:m schedule_gtfs_dataset_key",
        "right_only": "1 schedule_gtfs_dataset_key: m organization_name",
        "both": "m organization_name: m schedule_gtfs_datset_key",
    }
    m1._merge = m1._merge.map(indicator_values)

    # Re filter and save out to YML for each combo
    # One `organization_name` to many `schedule_gtfs_dataset_key`
    one_key_many_orgs_df = m1.loc[
        m1._merge == "1 schedule_gtfs_dataset_key: m organization_name"
    ]

    # Save to yml
    df_to_yaml(
        df=one_org_m_keys_df,
        nest1_column="organization_name",
        nest2_column="name",
        SITE_YML="one_org_many_keys.yml",
        title="1 organization_name: m schedule_gtfs_dataset-key, all values below are encompassed under one organization_name",
    )

    # One `organization_name` to many `schedule_gtfs_dataset_key`
    one_org_m_keys_df = m1.loc[
        m1._merge == "1 organization_name:m schedule_gtfs_dataset_key"
    ]
    # Save to yml
    df_to_yaml(
        df=one_key_many_orgs_df,
        nest1_column="name",
        nest2_column="organization_name",
        SITE_YML="one_key_many_orgs.yml",
        title="1 schedule_gtfs_dataset_key:m organization_name, only the 1st value is displayed in the portfolio",
    )
    # Many organization_name to many schedule_gtfs_datset_keys"
    m_org_m_keys_df = m1.loc[
        m1._merge == "m organization_name: m schedule_gtfs_datset_key"
    ]
    # Save to yml
    df_to_yaml(
        df=m_org_m_keys_df,
        nest1_column="organization_name",
        nest2_column="name",
        SITE_YML="many_keys_many_orgs.yml",
        title="m schedule_gtfs_dataset_key:m organization_name",
    )

In [29]:
generate_key_org_ymls(schd_vp_df)

Saved to yml
Saved to yml
Saved to yml


### One `organization_name` to many `schedule_gtfs_dataset_key`

In [ ]:
one_org_m_keys_list = count_orgs(
    schd_vp_df,
    "organization_name",
    "schedule_gtfs_dataset_key",
)

In [ ]:
# Filter
one_org_m_keys_df = schd_vp_df.loc[
    schd_vp_df.organization_name.isin(one_org_m_keys_list)
].drop(columns=["sched_rt_category"])

In [ ]:
one_org_m_keys_df.sort_values(by=["organization_name"])

### One `schedule_gtfs_dataset_key` to many `organization_name`

In [ ]:
one_key_many_orgs_list = count_orgs(
    schd_vp_df,
    "schedule_gtfs_dataset_key",
    "organization_name",
)

In [ ]:
# Filter
one_key_many_orgs_df = schd_vp_df.loc[
    schd_vp_df.schedule_gtfs_dataset_key.isin(one_key_many_orgs_list)
]

In [ ]:
schd_vp_df.loc[schd_vp_df.organization_name == "Basin Transit"]

In [ ]:
schd_vp_df.loc[
    schd_vp_df.schedule_gtfs_dataset_key == "1770249a5a2e770ca90628434d4934b1"
]

In [ ]:
schd_vp_df.loc[schd_vp_df.organization_name == "Palo Verde Valley Transit Agency"]

### Tag

In [ ]:
m1 = pd.merge(
    one_org_m_keys_df,
    one_key_many_orgs_df,
    on=["schedule_gtfs_dataset_key", "organization_name", "name"],
    how="outer",
    indicator=True,
)

In [ ]:
m1[["schedule_gtfs_dataset_key", "organization_name", "name", "_merge"]].sort_values(
    by=["_merge"]
)

In [ ]:
indicator_values = {
    "left_only": "1 organization_name:m schedule_gtfs_dataset_key",
    "right_only": "1 schedule_gtfs_dataset_key: m organization_name",
    "both": "m organization_name: m schedule_gtfs_datset_key",
}

In [ ]:
m1._merge = m1._merge.map(indicator_values)

In [ ]:
m1._merge.value_counts()

In [ ]:
def df_to_yaml(
    df: pd.DataFrame, nest1_column: str, nest2_column: str, SITE_YML: str, title: str
):
    """
    Dump Pandas Dataframe to a YAML.

    Parameters:
    df (pd.DataFrame): DataFrame with 'sched_rt_category' and 'organization_name' columns.
    title (str): Title to be added at the top of the YAML file.

    Returns:
    yaml_str (str): YAML string representation of the input DataFrame.
    """
    # Initialize an empty dictionary to store the result
    result = {}

    # Iterate over unique 'sched_rt_category' values
    for category in df[nest1_column].unique():
        # Filter the DataFrame for the current category
        category_df = df[df[nest1_column] == category]

        # Create a list of 'organization_name' values for the current category
        organization_names = category_df[nest2_column].tolist()

        # Add the category and organization names to the result dictionary
        result[category] = organization_names

    # Save to YML
    with open(SITE_YML, "w") as f:
        f.write(f"# {title}\n\n")
        output = yaml.dump(result, default_flow_style=False)
        f.write(output)
    print("Saved to yml")
    return result

In [ ]:
test = df_to_yaml(
    one_key_many_orgs_df,
    "name",
    "organization_name",
    "one_key_many_orgs.yml",
    "1 schedule_gtfs_dataset_key:m organization_name, only 1st value is displayed in the portfolio",
)

In [ ]:
test2 = df_to_yaml(
    one_org_m_keys_df,
    "organization_name",
    "name",
    "one_org_many_keys.yml",
    "1 organization_name: m _schedule_gtfs_dataset-key, all values below are encompassed in org_name",
)

## YML 2: Which operators we exclude 
* Explain whether it's due to 1 schedule_gtfs_dataset_key:m organization_name or we prefer another name

In [30]:
import deploy_portfolio_yaml

In [35]:
def generate_excluded_orgs_yml(df: pd.DataFrame):
    """
    Generate YML for excluded operators and explain why
    they aren't included.
    """
    # Manual list of operators we exclude
    operators_to_exclude = ["City of Alameda"]
    
    # Load in dataframe of organization_name values we display in our GTFS
    # Digest operator grain portfolio 
    orgs_in_portfolio = deploy_portfolio_yaml.generate_operator_grain_yaml()
    
    # Merge
    m1 = pd.merge(df, orgs_in_portfolio, how="outer", indicator=True)
    
    # Any left only values are excluded organizations 
    excluded_orgs = m1.loc[m1._merge == "left_only"]
    
    # Map dictionary of excluded operators
    excluded_orgs["reason_for_exclusion"] = excluded_orgs["organization_name"].map(
        reason_for_exclusion
    )
    
    # Subset
    excluded_orgs = excluded_orgs[["organization_name", "reason_for_exclusion"]]
    
    # Any organization without a manual value in the reason_for_exclusion dictionary
    # is excluded because it has another organization_name values that came before it 
    excluded_orgs = excluded_orgs.fillna(
        "1 schedule_gtfs_dataset_key:m organization_name, only 1st organization_name by alphabetical order is displayed in the portfolio"
    )

    # Generate YML
    df_to_yaml(
        df=excluded_orgs,
        nest1_column="reason_for_exclusion",
        nest2_column="organization_name",
        SITE_YML="excluded_orgs.yml",
        title="organization_name values that are excluded from the GTFS Digest portfolio",
    )

In [36]:
generate_excluded_orgs_yml(schd_vp_df)

Saved to yml


/tmp/ipykernel_1153/2174016483.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excluded_orgs["reason_for_exclusion"] = excluded_orgs["organization_name"].map(


## YML 3: Operators who have RT, Schedule, or Both

In [37]:
def generate_org_gtfs_status_yml(df: pd.DataFrame):
    # Subset
    df2 = df[
        [
            "sched_rt_category",
            "organization_name",
        ]
    ]
    # Generate YML 
    df_to_yaml(
        df2,
        "sched_rt_category",
        "organization_name",
        "org_gtfs_status.yml",
        "Operators who have RT (vp_only), Schedule, or Both (schedule_and_vp)",
    )

In [38]:
generate_org_gtfs_status_yml(schd_vp_df)

Saved to yml
